In [88]:
import pandas as pd
import re
import numpy as np
import requests
from datetime import datetime, timedelta
from helpers.helpers import load_data
from helpers.helpers import compute_delta_factors
from helpers.helpers import convert_to_hs
from helpers.helpers import clean_profit
from helpers.helpers import asic_price
import openmeteo_requests
import requests_cache
from retry_requests import retry
import os
from datetime import datetime, time

In [89]:
tickers = ['XMR-USD']
tickers

['XMR-USD']

In [90]:
df = load_data(tickers)

[*********************100%***********************]  1 of 1 completed


In [91]:
priceDict = {}
for ticker in [s.split("-")[0] for s in tickers]:
    priceDict[ticker] =  df.tail(7)[ticker].mean()

In [92]:
priceDict

{'XMR': np.float64(274.167977469308)}

In [93]:
with open('data/coinmarketcap_api_key.txt', 'r') as file:
    coinmarketcap_api_key = file.read()

In [94]:
coinmarketcap_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
params = {"symbol": "XMR"}
headers = {"X-CMC_PRO_API_KEY": coinmarketcap_api_key, "Accept": "application/json"}

response = requests.get(coinmarketcap_url, headers=headers, params=params)
data = response.json()

# Extract prices
coin_dict = {
    symbol: {
        "price": priceDict[symbol],
        "network_hashrate": data["data"][symbol].get("network_hashrate", "N/A"),
        "reward_block": data["data"][symbol].get("reward_block","N/A"),
        "upward_factor": data["data"][symbol].get("upward_factor","N/A"),
        "downward_factor": data["data"][symbol].get("downward_factor","N/A"),
    }
    for symbol in params["symbol"].split(",")
}

print(coin_dict)

{'XMR': {'price': np.float64(274.167977469308), 'network_hashrate': 'N/A', 'reward_block': 'N/A', 'upward_factor': 'N/A', 'downward_factor': 'N/A'}}


In [95]:
response = requests.get("https://api.octa.space/v1/network")
data = response.json()

In [96]:
minerstat_url = "https://api.minerstat.com/v2/coins"

response = requests.get(minerstat_url)
data = response.json()

# Extract network hashrate for specific coins
coins_of_interest = ["XMR"]

for coin_of_interest in coins_of_interest:
    coin_data = next((coin for coin in data if coin.get("coin") == coin_of_interest), None)
    if coin_data:  # Ensure the coin was found before updating
        coin_dict[coin_of_interest]["network_hashrate"] = coin_data["network_hashrate"]
        coin_dict[coin_of_interest]["reward_block"] = coin_data["reward_block"]
        coin_dict[coin_of_interest]["upward_factor"] = compute_delta_factors(df[coin_of_interest])[0]
        coin_dict[coin_of_interest]["downward_factor"] = compute_delta_factors(df[coin_of_interest])[1]
print(coin_dict)

{'XMR': {'price': np.float64(274.167977469308), 'network_hashrate': 4501831767.55, 'reward_block': 0.60235808, 'upward_factor': np.float64(1.0340412520192273), 'downward_factor': np.float64(0.967079406210581)}}


In [97]:
df = pd.read_csv("data/computers.csv")

In [98]:
df['Hashrate_raw'] = df['Hashrate']
df['Efficiency'] = df['Hashrate_raw'] / df['Power']

In [99]:
algorithms = [
    df['Algorithm'] == 'SHA-256',
    df['Algorithm'] == 'Blake3',
    df['Algorithm'] == 'Scrypt',
    df['Algorithm'] == 'KHeavyHash',
    df['Algorithm'] == 'Cuckatoo32'
]

In [100]:
df['Network'] = df['Algorithm'].apply(lambda x: 
    ["ALPH"] if x == 'Blake3' else 
    ['DOGE', 'LTC'] if x == 'Scrypt' else
    ["KAS"] if x == 'KHeavyHash' else 
    ["BTC"] if x == 'SHA-256' else
    ["OCTA"] if x == 'EtHash' else
    ["MWC"] if x == 'Cuckatoo32' else
    ["RXD"] if x == 'SHA512256d' else
    ["XMR"] if x == 'RandomX' else            
    ['None']
)

open-meto template here
https://open-meteo.com/en/docs?forecast_days=1&current=temperature_2m&temperature_unit=fahrenheit

In [101]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

with open('data/latitude.txt', 'r') as file:
    latitude = file.read()
with open('data/longitude.txt', 'r') as file:
    longitude = file.read()

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": latitude,
	"longitude": longitude,
	"current": "temperature_2m",
	"forecast_days": 1,
	"temperature_unit": "fahrenheit"
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
current = response.Current()
current_temperature_2m = current.Variables(0).Value()

In [102]:
with open('data/base_rate.txt', 'r') as file:
    base_rate = file.read()
with open('data/peak_rate.txt', 'r') as file:
    peak_rate = file.read()
with open('data/peak_start.txt', 'r') as file:
    peak_start = file.read()
with open('data/peak_end.txt', 'r') as file:
    peak_end = file.read()
with open('data/temp_cutoff.txt', 'r') as file:
    temp_cutoff = file.read()
    
base_rate = float(base_rate)
peak_rate = float(peak_rate)
peak_start = int(peak_start)
peak_end = int(peak_end)
temp_cutoff = int(temp_cutoff)

if (datetime.now().time() > time(peak_start, 0)) & (datetime.now().time() < time(peak_end, 0)):
    if current_temperature_2m < temp_cutoff:
        df['Expected_Price'] = df.apply(lambda row: asic_price(row['Hashrate_raw'], coin_dict, row['Power'], peak_rate * .7, 1, row['Network']), axis=1)
    else:
        df['Expected_Price'] = df.apply(lambda row: asic_price(row['Hashrate_raw'], coin_dict, row['Power'], peak_rate, 1, row['Network']), axis=1)
else:
    if current_temperature_2m < temp_cutoff:
        df['Expected_Price'] = df.apply(lambda row: asic_price(row['Hashrate_raw'], coin_dict, row['Power'], base_rate * .7, 1, row['Network']), axis=1)
    else:
        df['Expected_Price'] = df.apply(lambda row: asic_price(row['Hashrate_raw'], coin_dict, row['Power'], base_rate, 1, row['Network']), axis=1)

In [103]:
try: 
    electrical_usage_log = pd.read_csv("electrical_usage_log")
    newest_entry = df[['Model','Power','Expected_Price']]
    newest_entry.index = pd.Series([datetime.now()] * len(newest_entry))
    electrical_usage_log = pd.concat([electrical_usage_log, newest_entry], ignore_index=False)
except:
    electrical_usage_log = df[['Model','Power','Expected_Price']]
    electrical_usage_log.index = pd.Series([datetime.now()] * len(electrical_usage_log))

electrical_usage_log.to_csv("data/electrical_usage_log.csv")

In [104]:
#currently implemented for single device only
if df['Expected_Price'].item() > 0:
    expected_return = str(1)
else:
    expected_return = str(0)

In [72]:
with open('data/path.txt', 'r') as file:
    file_path = file.read()
with open(os.path.abspath(file_path), 'w') as f:
    f.write(expected_return)